<a href="https://colab.research.google.com/github/ynusinovich/streamflow-prediction/blob/main/ARE/HYSETS_DARTS_subset_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Streamflow Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import xarray as xr
import pandas as pd
import numpy as np

import torch  # maybe necessary because of version conflict?

In [3]:
# !pip install pyyaml==5.4.1
!pip install torch==1.8.1  # there appears to be a version conflict with the current version of pytorch and cuDNN
!pip install darts==0.21.0
# !pip install -U matplotlib==3.1.3
# !pip install cudnn==8.3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
pytorch-lightning 1.7.7 requires torch>=1.9.*, but you have torch 1.8.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1


In [4]:
#@title Load Watershed Properties and Data

In [5]:
# load static properties
lcrop = True

# load properties
if lcrop:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned_prairies.txt')
else:
    wsp = pd.read_csv(data_root+'/HYSETS_watershed_properties_cleaned.txt')
# some cleaning
wsp.drop(columns='Unnamed: 0', inplace=True)
wsp['watershed'] = wsp['Watershed_ID']
wsp.set_index('watershed', drop=True, inplace=True)
wsp

,Watershed_ID,Source,Name,Official_ID,Centroid_Lat_deg_N,Centroid_Lon_deg_E,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,...,Land_Use_Crops_frac,Land_Use_Snow_Ice_frac,Flag_Land_Use_Extraction,Permeability_logk_m2,Porosity_frac,Flag_Subsoil_Extraction,Hydrometric_station_latitude,Hydrometric_station_longitude,Flag_Natural_Boundaries,Flag_ALL
watershed,,,,,,,,,,,,,,,,,,,,,
865,865,HYDAT,WILLOW CREEK AT HIGHWAY NO. 811,05AB046,49.75476,-113.40730,2509.8584,1262.0,5.7682,1.6076,...,0.2563,0.0000,1,-15.262326,0.186644,1,49.75476,-113.40730,1,1
867,867,HYDAT,LITTLE BOW RIVER ABOVE TRAVERS RESERVOIR,05AC034,50.20417,-112.97780,3490.0697,1034.6,1.2383,1.8040,...,0.8288,0.0000,1,-15.200000,0.190000,1,50.20417,-112.97780,1,1
868,868,HYDAT,LITTLE BOW RIVER AT HIGHWAY NO. 533,05AC930,50.35417,-113.54310,791.6734,1026.4,0.6497,1.4250,...,0.9049,0.0000,1,-15.200000,0.190000,1,50.35417,-113.54310,1,1
869,869,HYDAT,WATERTON RIVER NEAR WATERTON PARK,05AD003,49.11366,-113.83940,612.6766,1838.1,22.5613,1.6875,...,0.0011,0.0057,1,-15.231325,0.106944,1,49.11366,-113.83940,1,1
877,877,HYDAT,BELLY RIVER NEAR MOON RIVER ROAD,05AD934,49.72870,-113.12470,3584.4565,1382.8,8.0552,1.7799,...,0.4126,0.0022,1,-15.359595,0.160534,1,49.72870,-113.12470,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,HYDAT,NETLEY CREEK NEAR PETERSFIELD,05OJ008,50.32722,-97.04417,641.0558,256.3,0.2838,1.2551,...,0.5614,0.0000,1,-11.800000,0.060000,1,50.32722,-97.04417,1,1
1273,1273,HYDAT,ICELANDIC RIVER NEAR RIVERTON,05SC002,50.96472,-97.03747,1236.7307,252.9,0.2952,1.6384,...,0.5162,0.0000,1,-11.803577,0.060137,1,50.96472,-97.03747,1,1
1274,1274,HYDAT,FISHER RIVER NEAR DALLAS,05SD003,51.35617,-97.51222,1706.3842,258.5,0.4215,1.5097,...,0.2668,0.0000,1,-11.800000,0.060000,1,51.35617,-97.51222,1,1


In [6]:
# load NetCDF file lazily
if lcrop:
    xds = xr.open_dataset(data_root+'/HYSETS_2020_QC_stations_mod2_crop.nc', chunks={'watershed':1})
else:
    xds = xr.open_dataset(data_root+'/HYSETS_2020_QC_stations_mod2.nc', chunks={'watershed':1})
# use watershed IDs to slice out watersheds of interest
xds = xds.sel(watershed=wsp['Watershed_ID'].values)
# load properties into memory
for var in xds.data_vars.values():
    if var.ndim == 1:
        var.load()
xds

<xarray.Dataset>
Dimensions:                     (time: 25202, watershed: 112)
Coordinates:
  * time                        (time) datetime64[ns] 1950-01-01 ... 2018-12-31
  * watershed                   (watershed) int32 865 867 868 ... 1274 1283 2365
Data variables: (12/15)
    watershedID                 (watershed) float64 865.0 867.0 ... 2.365e+03
    drainage_area               (watershed) float64 2.51e+03 ... 1.97e+03
    drainage_area_GSIM          (watershed) float64 nan nan ... nan 1.995e+03
    flag_GSIM_boundaries        (watershed) float64 0.0 0.0 0.0 ... 0.0 0.0 1.0
    flag_artificial_boundaries  (watershed) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    centroid_lat                (watershed) float64 49.75 50.2 ... 53.82 49.02
    ...                          ...
    discharge                   (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    pr                          (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmax                      (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmin                      (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    data_mask                   (watershed, time) bool dask.array<chunksize=(1, 25202), meta=np.ndarray>
    mask_300                    (watershed, time) uint16 dask.array<chunksize=(1, 25202), meta=np.ndarray>
Attributes: (12/13)
    title:          Hydrometeorological Sandbox of the École de technologie s...
    summary:        Hydrometeorological Sandbox of the École de technologie s...
    institution:    Hydrology, Climate and Climate Change Laboratory (HC3) at...
    institute_id:   HC3-ETS
    contact:        Richard Arsenault: richard.arsenault@etsmtl.ca
    date_created:   2020-05-26
    ...             ...
    featureType:    timeSeries
    cdm_data_type:  station
    license:        ODC-BY
    keywords:       HYSETS, GHCND, GSIM, hydrology, North America, streamflow...
    conventions:    CF-1.6, ACDD-1.3
    activity:       HYSETS

In [7]:
#@title Extract Periods of Consecutive Valid Data

In [8]:
data_mask = xds.data_mask
mask_300 = xds.mask_300

# some reorganization of data
ts_varnames = ('discharge', )
cov_varnames = ('pr', 'tasmin', 'tasmax')
ts_xds = xr.Dataset({varname:xds[varname] for varname in ts_varnames+cov_varnames})
ts_xds

<xarray.Dataset>
Dimensions:    (time: 25202, watershed: 112)
Coordinates:
  * time       (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2018-12-31
  * watershed  (watershed) int32 865 867 868 869 877 ... 1273 1274 1283 2365
Data variables:
    discharge  (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    pr         (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmin     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>
    tasmax     (watershed, time) float32 dask.array<chunksize=(1, 25202), meta=np.ndarray>

In [9]:
# determine means and variances by watershed for static covariats and normalization
sds = xr.Dataset(data_vars=None, coords={'watershed':xds.watershed})
for varname in ['discharge', 'pr', 'tasmin', 'tasmax']:
    data_var = xds[varname]
    # compute means/std
    sds[varname+'_mean'] = data_var.mean(dim=('time',)).load()
    sds[varname+'_std']  = data_var.std(dim=('time',)).load()

# add to properties dataframe
stats_df = sds.to_dataframe()
stats_df

,discharge_mean,discharge_std,pr_mean,pr_std,tasmin_mean,tasmin_std,tasmax_mean,tasmax_std
watershed,,,,,,,,
865,6.212508,16.222912,1.397961,3.986137,-2.299139,10.016271,10.625340,11.139233
867,2.730594,3.131016,1.212713,3.467988,-2.114604,10.326642,11.219891,11.972828
868,1.965355,2.361048,1.236688,3.745683,-2.588431,10.382421,10.750421,12.201200
869,17.894640,26.214912,2.954037,6.186049,-1.649298,9.022515,9.862430,10.689588
877,27.447258,46.197613,1.963749,4.449354,-1.377792,9.549591,10.827139,11.038117
...,...,...,...,...,...,...,...,...
1230,1.662630,5.325261,1.434288,4.343831,-3.432139,14.134577,7.724002,14.932825
1273,3.228191,10.444004,1.395137,4.093546,-4.080882,14.225872,7.342315,15.035342
1274,2.721829,8.511021,1.419544,4.080548,-4.674377,14.303836,7.375019,15.017003


In [10]:
## assemble static dataframes with means & std
# join with existing properties by index
wsp_ms = wsp.join(stats_df, on=None, how='left')
# validate
assert (np.isnan(wsp_ms['discharge_mean'].values)).any() == False

# dropable columns
str_cols = ['Source', 'Name', 'Official_ID']
flag_cols = ['Flag_ALL', 'Flag_Shape_Extraction', 'Flag_Terrain_Extraction', 'Flag_Land_Use_Extraction', 'Flag_Subsoil_Extraction', 'Flag_Natural_Boundaries']
drop_cols = ['Watershed_ID', 'Centroid_Lon_deg_E', 'Hydrometric_station_latitude', 'Hydrometric_station_longitude'] + flag_cols + str_cols

# trim props for static covariates
print('\nDropping columns for traing data:\n', *drop_cols)
wsp_train = wsp_ms.drop(columns=drop_cols, inplace=False)
wsp_train


Dropping columns for traing data:
 Watershed_ID Centroid_Lon_deg_E Hydrometric_station_latitude Hydrometric_station_longitude Flag_ALL Flag_Shape_Extraction Flag_Terrain_Extraction Flag_Land_Use_Extraction Flag_Subsoil_Extraction Flag_Natural_Boundaries Source Name Official_ID


,Centroid_Lat_deg_N,Drainage_Area_km2,Elevation_m,Slope_deg,Gravelius,Perimeter,Aspect_deg,Land_Use_Forest_frac,Land_Use_Grass_frac,Land_Use_Wetland_frac,...,Permeability_logk_m2,Porosity_frac,discharge_mean,discharge_std,pr_mean,pr_std,tasmin_mean,tasmin_std,tasmax_mean,tasmax_std
watershed,,,,,,,,,,,,,,,,,,,,,
865,49.75476,2509.8584,1262.0,5.7682,1.6076,283.798,118.6022,0.2480,0.3854,0.0077,...,-15.262326,0.186644,6.212508,16.222912,1.397961,3.986137,-2.299139,10.016271,10.625340,11.139233
867,50.20417,3490.0697,1034.6,1.2383,1.8040,375.464,105.8134,0.0079,0.0923,0.0029,...,-15.200000,0.190000,2.730594,3.131016,1.212713,3.467988,-2.114604,10.326642,11.219891,11.972828
868,50.35417,791.6734,1026.4,0.6497,1.4250,141.224,163.4672,0.0009,0.0117,0.0003,...,-15.200000,0.190000,1.965355,2.361048,1.236688,3.745683,-2.588431,10.382421,10.750421,12.201200
869,49.11366,612.6766,1838.1,22.5613,1.6875,147.343,287.0749,0.5341,0.2608,0.0026,...,-15.231325,0.106944,17.894640,26.214912,2.954037,6.186049,-1.649298,9.022515,9.862430,10.689588
877,49.72870,3584.4565,1382.8,8.0552,1.7799,375.794,61.3698,0.2013,0.2598,0.0158,...,-15.359595,0.160534,27.447258,46.197613,1.963749,4.449354,-1.377792,9.549591,10.827139,11.038117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,50.32722,641.0558,256.3,0.2838,1.2551,111.934,112.7949,0.1996,0.1074,0.0525,...,-11.800000,0.060000,1.662630,5.325261,1.434288,4.343831,-3.432139,14.134577,7.724002,14.932825
1273,50.96472,1236.7307,252.9,0.2952,1.6384,202.881,78.8539,0.2345,0.1281,0.0277,...,-11.803577,0.060137,3.228191,10.444004,1.395137,4.093546,-4.080882,14.225872,7.342315,15.035342
1274,51.35617,1706.3842,258.5,0.4215,1.5097,219.568,49.3099,0.4043,0.1775,0.0465,...,-11.800000,0.060000,2.721829,8.511021,1.419544,4.080548,-4.674377,14.303836,7.375019,15.017003


In [18]:
%%time
# loop over stations
from darts import TimeSeries

verbosity = 1

test_ratio = 0.3

ts_series_train = []
cov_series_train = []
ws_yrs_train = 0
ts_series_test = []
cov_series_test = []
ws_yrs_test = 0

# for wid in [1151]: 
# for wid in [915]: 
for wid in xds.watershed.values:

    ## identify indices pf valid periods
    ws_days = xds.mask_300.sel(watershed=wid).values
    wsd_del = np.diff(np.where(ws_days > 0, np.int8(1), np.int8(0),),)  # potentially larger arrays
    # find start points of valid periods
    start_idxs = np.nonzero(wsd_del > 0)[0] + 1
    if ws_days[0] > 0: start_idxs =np.concatenate(([0],start_idxs))
    # find end points of valid periods
    end_idxs = np.nonzero(wsd_del < 0)[0] + 1
    if ws_days[-1] > 0: end_idxs = np.concatenate((end_idxs,[ws_days.size]))
    assert end_idxs.size == start_idxs.size
    # verify length of each period
    prd_len = end_idxs - start_idxs
    #print(wid, prd_len)
    if verbosity and prd_len.size > 0: 
        if verbosity == 1: print('ID:', wid, 'Periods:', prd_len)
        else: print("\nWatershed ID:", wid)
        #print(wid, start_idxs, end_idxs)
    assert ( prd_len >= 300 ).all()
    # test even more stringently
    test_idxs = np.stack((start_idxs, end_idxs), axis=1)
    assert ( np.diff( test_idxs.flatten() ) > 0 ).all(), test_idxs

    ## Generate Timeseries objects
    # select by watershed
    ts_ws = ts_xds.sel(watershed=wid)
    stat_ws = wsp_train.loc[wid]  # indexed by watershed ID
    
    # loop over time periods and generate TimeSeries
    for n,si,ei in zip(range(1,len(start_idxs)+1),start_idxs, end_idxs):
        if verbosity > 1: print('Period:',si,ei)

        # slice time
        tsx = ts_ws.isel(time=slice(si,ei))
        dti = pd.DatetimeIndex(tsx.time)
        # extract arrays and normalize
        ts_vars = dict()
        for varname,ts_var in tsx.data_vars.items():
            ts_vars[varname] = ( ts_var.values - stat_ws[varname+'_mean'] ) / stat_ws[varname+'_std']
        ts_vars['dayofyear'] = -1.415 * np.cos(dti.day_of_year.values * 2*np.pi / 365.25, dtype=np.float32)  # sort of normalize

        # create TimeSeries
        ts_data = np.stack([ts_vars[varname] for varname in ts_varnames], axis=1)
        ts_series = TimeSeries.from_times_and_values(dti, ts_data, static_covariates=stat_ws)
        cov_data = np.stack([ts_vars[varname] for varname in cov_varnames+('dayofyear',)], axis=1)
        cov_series = TimeSeries.from_times_and_values(dti, cov_data, static_covariates=stat_ws)
        # assign to train or test
        ts_len = len(dti)
        if ts_len < 10000:
            if n % 2 == 0:                
                ts_series_test.append(ts_series)
                cov_series_test.append(cov_series)
                ws_yrs_test += ts_len
            else:
                ts_series_train.append(ts_series)
                cov_series_train.append(cov_series)
                ws_yrs_train += ts_len
        else:
            split_idx = int( test_ratio * ts_len)
            ts_series_test.append(ts_series[:split_idx])
            cov_series_test.append(cov_series[:split_idx])
            ws_yrs_test += split_idx  # length of the first part
            ts_series_train.append(ts_series[split_idx:])
            cov_series_train.append(cov_series[split_idx:])
            ws_yrs_train += (ts_len - split_idx)  # length of the second part

test_ratio = ws_yrs_test/(ws_yrs_train+ws_yrs_test)
print(f"\nGenerated {len(ts_series_train)} training and {len(ts_series_test)} test TimeSeries objects with Covariates,\n" +
      f"for a total of {ws_yrs_train/365.25:.0f} and {ws_yrs_test/365.25:.0f} watershed-years, respectively ({int(test_ratio*100):d}% test).")

ID: 869 Periods: [24106]
ID: 911 Periods: [1157 3608]
ID: 915 Periods: [2952]
ID: 933 Periods: [17473   365]
ID: 992 Periods: [1430]
ID: 993 Periods: [17113]
ID: 1022 Periods: [16015]
ID: 1023 Periods: [1402]
ID: 1053 Periods: [13437  1113]
ID: 1089 Periods: [8474]
ID: 1110 Periods: [3731]
ID: 1113 Periods: [13545]
ID: 1114 Periods: [9437]
ID: 1151 Periods: [18028  2828  1081]
ID: 1153 Periods: [5146]
ID: 1160 Periods: [16802]
ID: 1161 Periods: [8187]
ID: 1182 Periods: [7550]
ID: 1188 Periods: [8707]
ID: 1193 Periods: [9439]
ID: 1198 Periods: [20760]
ID: 1229 Periods: [2922]
ID: 1274 Periods: [13067]
ID: 1283 Periods: [ 784 1154 1011  375 6904 1739  758  494  501  999  628]

Generated 30 training and 19 test TimeSeries objects with Covariates,
for a total of 552 and 175 watershed-years, respectively (24% test).
CPU times: user 1.97 s, sys: 70.7 ms, total: 2.04 s
Wall time: 1.95 s


In [19]:
cov_data.dtype

dtype('float32')

In [12]:
#@title Train a LSTM Model with the Data


In [ ]:
## load existing LSTM model
from darts.models import RNNModel

VERSION = 1
# N = 1

filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.pt"

model = RNNModel.load(filepath)

In [22]:
## Create a new LSTM model

# from darts.models import RNNModel

# VERSION = 1

# # simplified version of the Kratzert model
# model = RNNModel(
#     model="LSTM",
#     input_chunk_length=270,
#     #training_length=280,
#     hidden_dim=128, # 256
#     n_rnn_layers=1,
#     dropout=0.1, # 0.4
#     n_epochs=10,
#     pl_trainer_kwargs={"accelerator": "gpu", "gpus": [0]}
# )

# # train model for one epoch
# model.fit(ts_series_train, future_covariates=cov_series_train, verbose=True)

# N = 0
# filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.pt"
# model.save(filepath)
# print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive   ***   \n",f"'{filepath}'")

In [ ]:
# train model for some epochs
for n in range(10):
    N += 1
    print(f"   ***   Training Iteration {N}   ***")
    model.fit(ts_series_train, future_covariates=cov_series_train, verbose=True)
    filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.pt"
    model.save(filepath)
    print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive   ***   \n",f"'{filepath}'")

   ***   Training Iteration 1   ***


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training: 0it [00:00, ?it/s]

In [26]:
# last loss: loss=0.0101, train_loss=0.0157

VERSION = 1
N = 0
filepath = data_root+f"/HYSETS_LSTM_v{VERSION}.{N}.pt"
model.save(filepath)
print(f"\n   ***   Saved trained model version {VERSION}.{N} to Google Drive   ***   \n",f"'{filepath}'")


   ***   Saved trained model version 1.0 to Google Drive   ***   
 '/content/drive/My Drive/Streamflow Data//HYSETS_LSTM_v1.0.pt'
